In [4]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

# %%
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# stop being lazy (no!)
# spark = (SparkSession.builder
#                      .config("spark.sql.repl.eagerEval.enabled", "True")
#                      .getOrCreate())

# %%
#spark.sparkContext

# %%
sc = spark.sparkContext
sqlContext = spark

# %%
spark.sparkContext.setLogLevel("ERROR")
'''
# %%
spark.read
dir(spark.read)

# spark.read.csv() === spark.read.format('csv').load()

# %%
spark.read?
spark.read??
print(spark.read.__doc__)
'''
# %%
book = spark.read.text("simple_map.csv")

book

# %%
book.printSchema()

# %%
book.show()

# %%
book.show(10, truncate=50)



root
 |-- value: string (nullable = true)

+------------+
|       value|
+------------+
| id,name,age|
|  1,Alice,30|
|    2,Bob,25|
|3,Charlie,35|
+------------+

+------------+
|       value|
+------------+
| id,name,age|
|  1,Alice,30|
|    2,Bob,25|
|3,Charlie,35|
+------------+



In [5]:
# %%
from pyspark.sql.functions import split

lines = book.select(split(book.value, " ").alias("line"))

lines.show(5)

# %%

# returns a dataframe
book.select(book.value)

# %%
from pyspark.sql.functions import col

book.select(book.value)
book.select(book["value"])
book.select(col("value"))
book.select("value")



+--------------+
|          line|
+--------------+
| [id,name,age]|
|  [1,Alice,30]|
|    [2,Bob,25]|
|[3,Charlie,35]|
+--------------+



DataFrame[value: string]

In [6]:
# %%
from pyspark.sql.functions import split

lines = book.select(split(col("value"), " "))

lines

# %%
lines.printSchema()

# %%
lines.show(5)

# %%
book.select(split(col("value"), " ")).printSchema()

# %%
book.select(split(col("value"), " ").alias("line")).printSchema()

# %%
lines = book.select(split(book.value, " ").alias("line"))
lines.show(5)

# %%
# lines = book.select(split(book.value, " "))
# lines = lines.withColumnRenamed("split(value,  )", "line")



root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)

+-------------------+
|split(value,  , -1)|
+-------------------+
|      [id,name,age]|
|       [1,Alice,30]|
|         [2,Bob,25]|
|     [3,Charlie,35]|
+-------------------+

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)

+--------------+
|          line|
+--------------+
| [id,name,age]|
|  [1,Alice,30]|
|    [2,Bob,25]|
|[3,Charlie,35]|
+--------------+



In [7]:
# %%
from pyspark.sql.functions import explode

words = lines.select(explode(col("line")).alias("word"))

words.show(20)

# %%
words.printSchema()

# %%
from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))

words_lower.show(100)

# %%
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    # regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
    regexp_extract(col("word_lower"),  r"(\W+)?([a-z]+)", 2).alias("word")
)

words_clean.show(100)

# %%
words_nonull = words_clean.where(col("word") != "")

words_nonull.show(100)


+------------+
|        word|
+------------+
| id,name,age|
|  1,Alice,30|
|    2,Bob,25|
|3,Charlie,35|
+------------+

root
 |-- word: string (nullable = false)

+------------+
|  word_lower|
+------------+
| id,name,age|
|  1,alice,30|
|    2,bob,25|
|3,charlie,35|
+------------+

+-------+
|   word|
+-------+
|     id|
|  alice|
|    bob|
|charlie|
+-------+

+-------+
|   word|
+-------+
|     id|
|  alice|
|    bob|
|charlie|
+-------+

